In [ ]:
# ✅ Install dependencies
!pip install -U tensorflow==2.15 tensorflow-privacy==0.9


In [ ]:
# ✅ Imports
import tensorflow as tf
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasAdamOptimizer
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# ✅ Load and preprocess MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0
x_train = x_train[..., tf.newaxis]
x_test  = x_test[..., tf.newaxis]

print("Training shape:", x_train.shape)
print("Test shape:", x_test.shape)


In [ ]:
# ✅ Build model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])


In [ ]:
# ✅ Compile with Differential Privacy
dp_optimizer = DPKerasAdamOptimizer(
    l2_norm_clip=1.0,
    noise_multiplier=1.1,
    num_microbatches=256,
    learning_rate=0.001
)

model.compile(
    optimizer=dp_optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


In [ ]:
# ✅ Train the model
history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=256,
    validation_split=0.1
)
